# ChemMind Quickstart

This notebook demonstrates training and evaluating a GNN on a MoleculeNet dataset and running a few predictions.


In [ ]:
import os
import yaml
import torch

from utils.data import build_dataloaders
from models.build import build_model
from utils.training import train_and_evaluate

config_path = os.path.join("..", "config.yaml")
with open(config_path, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, val_loader, test_loader, metadata = build_dataloaders(config)

model = build_model(
    model_name=config["model"]["name"],
    input_dim=metadata["num_node_features"],
    edge_dim=metadata.get("num_edge_features", 0),
    output_dim=metadata["num_tasks"],
    hidden_dim=config["model"]["hidden_dim"],
    num_layers=config["model"]["num_layers"],
    dropout=config["model"]["dropout"],
    use_batch_norm=config["model"].get("batch_norm", True),
    pooling=config["model"].get("pooling", "mean"),
).to(device)

results = train_and_evaluate(
    model,
    (train_loader, val_loader, test_loader),
    config,
    device,
    writer=None,
    task_type=config["task"],
    metadata=metadata,
)
results


In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from utils.featurization import smiles_to_mol, mol_to_graph

smiles_list = ["CCO", "CC(=O)OC1=CC=CC=C1C(=O)O"]
images = []
for s in smiles_list:
    mol = Chem.MolFromSmiles(s)
    images.append(Draw.MolToImage(mol))
images
